<a href="https://colab.research.google.com/github/gauthiermartin/pytorch-deep-learning-course/blob/main/01_pytorch_workflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorch Workflow

Let's explore an example PyTorch end-to-end workflow

Resources :
* Ground truth notebook - https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
* Book version of the notebook - https://www.learnpytorch.io/01_pytorch_workflow/

In [4]:
what_are_we_covering = {
    1: "data (prepare and load)",
    2: "build model",
    3: "fitting or model to the data (training)",
    4: "making predictions and evaluating a model (inference)",
    5: "save an load a model",
    6 : "putting it all together",
}

what_are_we_covering

{1: 'data (prepare and load)',
 2: 'build model',
 3: 'fitting or model to the data (training)',
 4: 'making predictions and evaluating a model (inference)',
 5: 'save an load a model',
 6: 'putting it all together'}

In [6]:
import torch
from torch import nn #nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.0.1+cu118'

## 1.Data (preparing and loading)

Data can be almost anything ... in machine learning

* Excel spreadsheet
* Image of any type
* Video
* Audio
* DNA
* Text

Machine Learning is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn patterns in that numerical representation.